You are given a rows x cols matrix grid representing a field of cherries where grid[i][j] represents the number of cherries that you can collect from the (i, j) cell.

You have two robots that can collect cherries for you:

Robot #1 is located at the top-left corner (0, 0), and
Robot #2 is located at the top-right corner (0, cols - 1).
Return the maximum number of cherries collection using both robots by following the rules below:

From a cell (i, j), robots can move to cell (i + 1, j - 1), (i + 1, j), or (i + 1, j + 1).
When any robot passes through a cell, It picks up all cherries, and the cell becomes an empty cell.
When both robots stay in the same cell, only one takes the cherries.
Both robots cannot move outside of the grid at any moment.
Both robots should reach the bottom row in grid.

# Brute Force

In [12]:
#   TODO: should remove the shape thing and use a class maybe to store the value
def get_valid_actions(field_shape: tuple, current_loc:tuple) -> list[tuple]:
    """
    shapes are in (i, j) format
    """
    result = []
    next_i = current_loc[0] + 1
    
    #   check i
    if next_i >= field_shape[0]:
        return result
    
    for step in range(-1, 2):
        next_j = current_loc[1] + step
        #   check j
        if next_j < 0 or next_j >= field_shape[1]:
            continue
        result.append((next_i, next_j))
    
    return result


In [ ]:
#   test
list_shape = (5,5)
locs = [(0,0), (0,4), (1,3)]
for x in locs:
    print(get_valid_actions(list_shape,x))

In [22]:
import copy
#   should start at a given loc
#   expand my next moves
#   store the path that im taking
list_shape = (5,5)
paths = []

def bfs(current_loc:tuple, path:list):
    path.append(current_loc)
    #   get next possible actions:
    next_actions = get_valid_actions(list_shape, current_loc)
    if not next_actions:
        #   termination
        paths.append(path)
        return
    
    for action in next_actions:
        #   important to use a new object and not the main reference
        bfs(action, copy.copy(path))

bfs((0,0),[])
for x in paths:
    print(x)

[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0)]
[(0, 0), (1, 0), (2, 0), (3, 0), (4, 1)]
[(0, 0), (1, 0), (2, 0), (3, 1), (4, 0)]
[(0, 0), (1, 0), (2, 0), (3, 1), (4, 1)]
[(0, 0), (1, 0), (2, 0), (3, 1), (4, 2)]
[(0, 0), (1, 0), (2, 1), (3, 0), (4, 0)]
[(0, 0), (1, 0), (2, 1), (3, 0), (4, 1)]
[(0, 0), (1, 0), (2, 1), (3, 1), (4, 0)]
[(0, 0), (1, 0), (2, 1), (3, 1), (4, 1)]
[(0, 0), (1, 0), (2, 1), (3, 1), (4, 2)]
[(0, 0), (1, 0), (2, 1), (3, 2), (4, 1)]
[(0, 0), (1, 0), (2, 1), (3, 2), (4, 2)]
[(0, 0), (1, 0), (2, 1), (3, 2), (4, 3)]
[(0, 0), (1, 1), (2, 0), (3, 0), (4, 0)]
[(0, 0), (1, 1), (2, 0), (3, 0), (4, 1)]
[(0, 0), (1, 1), (2, 0), (3, 1), (4, 0)]
[(0, 0), (1, 1), (2, 0), (3, 1), (4, 1)]
[(0, 0), (1, 1), (2, 0), (3, 1), (4, 2)]
[(0, 0), (1, 1), (2, 1), (3, 0), (4, 0)]
[(0, 0), (1, 1), (2, 1), (3, 0), (4, 1)]
[(0, 0), (1, 1), (2, 1), (3, 1), (4, 0)]
[(0, 0), (1, 1), (2, 1), (3, 1), (4, 1)]
[(0, 0), (1, 1), (2, 1), (3, 1), (4, 2)]
[(0, 0), (1, 1), (2, 1), (3, 2), (4, 1)]
[(0, 0), (1, 1),

In [23]:
class Solution:
    def isValid(self, s: str) -> bool:
        pairs = {
            '(':')',
            '{':'}',
            '[':']'
        }
        stack = []
        for i in range(len(s)):
            if s[i] in pairs.keys():
                #   push
                stack.append()
                continue

            #   else, pop
            last_opened = stack.pop()
            if pairs[last_opened] != s[i]:
                return False

        return True

SyntaxError: invalid syntax (4137536928.py, line 10)